# Dense ODE-Net 求解薛定谔方程

## 概述
薛定谔方程描述了封闭量子系统的演化。 最近的研究表明，现代神经网络的结钩如残差连接与微分方程的离散结构有相似之处：[Bridging Deep Architectures and Numerical Differential Equations](https://arxiv.org/pdf/1710.10121.pdf); [Neural Ordinary Differential Equations](https://arxiv.org/pdf/1806.07366.pdf). 神经网络不同层之间的连接与微分方程的离散结钩有着紧密的联系，如残差连接等同于恒等映射。在ResNet的基础上，DenseNet引入了更密集的Dense Connections, 能够获得更加复杂的网络拓扑结构。受这些研究的启发，这个demo为ODE-Net引入可学习的Dense Connections，用于求解薛定谔方程。

## 问题描述
本案例求解薛定谔方程的正问题，使用我们称之为Dense ODE-Net的网络学习微分方程离散结构，并实现量子态演化的长时间预测。
注：本案例中的哈密顿量均厄密且不含时，对于非厄密哈密顿量如PT对称的哈密顿量，本案例暂不考虑。

## 薛定谔方程
$$
i\hbar\dfrac{\partial|\Psi>}{\partial t} = H|\Psi>
$$

$$
H=H^\dagger
$$

## Dense ODE-Net的模型结构
Dense ODE-Net的深度为$depth$，所有层共享权重$H$，$H$由哈密顿量决定。网络中可学习的参数为不同层之间的dense connections的权重，从第i层到第j层的连接权重为$W_{i,j}$。其中$W_{0,depth}$固定为1，以保证从输入到输出之间的恒等映射。
![Cell](images/Dense_Cell.png)

通过设定恰当的深度与连接权重W,可以得到许多著名的差分方案。
如前向欧拉法：

![Forward Euler](images/Forward_Euler.png)

四阶Runge-Kutta法：

![Fourth_Runge_Kutta Euler](images/Fourth_Runge_Kutta.png)

## 技术路径
求解该问题的具体流程如下：

1. 构建模型。
2. 训练: 逐步增加训练数据的时间步数，进行训练。
3. 模型推理及可视化。

In [2]:
import mindspore as ms
import numpy as np
import os
from mindspore import nn
from mindspore.amp import all_finite
from src.dense_ode_net import DenseODENet, DenseODENetWithLoss
from src.data_generate import Generator
from utils import generate_train_data, init_env, makedir, get_config, evaluate, test

## 设置训练环境

In [2]:
ms.set_seed(1999)
np.random.seed(1999)
my_config = get_config('./config.yaml')
makedir(my_config)
my_config['device_target'] = 'CPU'
my_config['mode'] = 'pynative'
my_config['device_id'] = 0
ms.set_context(mode=ms.PYNATIVE_MODE, device_target=my_config['device_target'], device_id=my_config['device_id'])
print('----------------------------- Env settings -----------------------------')
print('running on {}, device id: {}, context mode: {}'.format(my_config['device_target'], my_config['device_id'],
                                                              my_config['mode']))

----------------------------- Env settings -----------------------------
running on CPU, device id: 0, context mode: pynative


## 构建模型
Dense ODE-Net 在类DenseODENet中定义，需要提供网络深度，哈密顿量维数，Dense Connections Weight的初始化范围，网络最大时间步长。

In [3]:
def init_model(config):
    model = DenseODENet(depth=config['depth'], h_dim=config['h_dim'], init_range=config['w_init_range'],
                        max_dt=config['max_dt'])
    return model

# 定义单步训练

In [4]:
def single_train(config, time_step: int, net_with_loss: DenseODENetWithLoss, data_generator):
    lr = config['lr'] * np.power(config['lr_reduce_gamma'], (time_step - 1) // config['lr_reduce_interval'])
    my_optimizer = nn.Adam(params=net_with_loss.ode_net.trainable_params(), learning_rate=lr)

    def forward_fn(h, trajectory, t_points):
        loss = net_with_loss.get_loss(H=h, batch_trajectories=trajectory, t_points=t_points)
        return loss
    value_and_grad = ms.ops.value_and_grad(forward_fn, None, weights=my_optimizer.parameters)

    def train_process(h, trajectory, t_points):
        loss, grads = value_and_grad(h, trajectory, t_points)
        if all_finite(grads):
            my_optimizer(grads)
        return loss

    train_dataset = generate_train_data(config=config, data_generator=data_generator, time_step=time_step)
    for epoch_idx in range(1, config['epochs'] + 1):
        net_with_loss.ode_net.set_train(mode=True)
        avg_loss = 0
        for H, trajectories, time_points in train_dataset.fetch():
            train_loss = train_process(H, trajectories, time_points)
            avg_loss += train_loss.asnumpy()
        print('time_step: {} -- epoch: {} -- lr: {} -- loss: {}'.format(time_step, epoch_idx, lr, avg_loss))

        # generate new data
        if epoch_idx % config['generate_data_interval'] == 0:
            train_dataset = generate_train_data(config=config, data_generator=data_generator, time_step=time_step)
    # save
    save_path = os.path.join(config['save_directory'], 'dense_ode_net_step{}.ckpt'.format(time_step))
    ms.save_checkpoint(net_with_loss.ode_net, save_path)
    # evaluate
    evaluate(config=config, model=net_with_loss.ode_net, data_generator=data_generator)
    print('-- Current Dense Weight')
    print(net_with_loss.ode_net.dense_w())
    return

## 训练

In [5]:
def train(config):
    data_generator = Generator(dim=config['h_dim'])
    dense_ode_net = init_model(config)
    net_with_loss = DenseODENetWithLoss(ode_net=dense_ode_net)
    # first evaluate
    evaluate(config=config, model=net_with_loss.ode_net, data_generator=data_generator)
    print('-- Current Dense Weight')
    print(net_with_loss.ode_net.dense_w())

    for time_step in range(1, config['time_step_num'] + 1):
        single_train(config=config, time_step=time_step, net_with_loss=net_with_loss, data_generator=data_generator)
    return

In [6]:
train(my_config)

H 1 test forward ...
H 2 test forward ...
H 3 test forward ...
H 4 test forward ...
H 5 test forward ...
H 6 test forward ...
H 7 test forward ...
H 8 test forward ...
H 9 test forward ...
H 10 test forward ...
=================================== evaluate ===================================
-- Max evaluate relative L2 error: 1.9590126276016235
-- Current Dense Weight
[[0.         0.9459363  0.9811951  0.9636938  1.        ]
 [0.         0.9143088  0.93390936 0.9901344  0.9833571 ]
 [0.         0.         0.9606229  0.9006332  0.9131826 ]
 [0.         0.         0.         0.9657586  0.98012316]
 [0.         0.         0.         0.         0.9815558 ]]
time_step: 1 -- epoch: 1 -- lr: 0.005 -- loss: 9.402620480614132e-05
time_step: 1 -- epoch: 2 -- lr: 0.005 -- loss: 7.33129054424353e-05
time_step: 1 -- epoch: 3 -- lr: 0.005 -- loss: 5.6141401728382334e-05
time_step: 1 -- epoch: 4 -- lr: 0.005 -- loss: 4.281160431673925e-05
time_step: 1 -- epoch: 5 -- lr: 0.005 -- loss: 3.25738128594821

## 测试随机哈密顿量预测精度

In [1]:
test(my_config, show_results=True)

NameError: name 'test' is not defined